<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/model/ATE_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('punkt')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Load Dataset

In [3]:
test = pd.read_json('/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/ATE_ABSITA_test_set/ate_absita_gold.ndjson'
                       , lines=True)

In [4]:
train = pd.read_json('/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/ATE_ABSITA_training_set/ate_absita_training.ndjson'
                       , lines=True)

In [5]:
train.head(3)

,sentence,id_sentence,score,polarities,aspects_position,aspects
0,"Ottimo prodotto di marca, la qualità é veramen...",4b7254a1-3f31-4143-ab22-a8558aa4a73b,5,"[[0, 0], [0, 1], [1, 0]]","[[120, 142], [71, 79], [29, 36]]","[provvisto di una tasca, capiente, qualità]"
1,Ottimo rasoio dal semplice utilizzo. Rade molt...,4b74d99d-891f-4526-bbd3-549fa244cd1c,5,"[[1, 0], [1, 0], [1, 0], [1, 0]]","[[18, 26], [37, 41], [79, 86], [99, 105]]","[semplice, Rade, Pratico, pulire]"
2,Un quarto delle dimensioni dello Show original...,4b7ff44f-fa9f-4ef0-97c8-e295e70ccc9b,5,"[[1, 0], [1, 0], [1, 0], [0, 0]]","[[118, 132], [51, 62], [65, 70], [16, 26]]","[modalità notte, prestazioni, suono, dimensioni]"


In [ ]:
print(test.columns)
print(train.columns)

Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')
Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')


In [6]:
train.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
test.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
print(f'Contains {len(train)} sentences')
print(f'Contains {len(test)} sentences')

Contains 3054 sentences
Contains 1200 sentences


In [7]:
train["review_type"] = train["score"].apply(lambda x: "neg" if x < 5 else "pos")
test["review_type"] = test["score"].apply(lambda x: "neg" if x < 5 else "pos")

In [8]:
print(f'TRAIN::\n{train.review_type.value_counts()}')
print(f'COMPLETO::\n{test.review_type.value_counts()}')

TRAIN::
pos    2150
neg     904
Name: review_type, dtype: int64
COMPLETO::
pos    857
neg    343
Name: review_type, dtype: int64


**Necessaria Augemntation**

In [9]:
train.drop(columns=['score'], inplace=True)
test.drop(columns=['score'], inplace=True)

Shuffled datset for improving the learning since negative entries are lower than positive

In [10]:
shuffle=train.sample(frac=1)

In [ ]:
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

changhe shuffle with train if u want to use standard db

In [11]:


one_hot_train = OneHotEncoder(sparse=False).fit_transform(
  shuffle.review_type.to_numpy().reshape(-1, 1)
)

one_hot_test = OneHotEncoder(sparse=False).fit_transform(
  test.review_type.to_numpy().reshape(-1, 1)
)

Trasforma da pos, neg a [0,1], [1,0]

In [127]:


matrix = [word_tokenize(sent) for sent in train.sentence]

for elem in matrix:
  if len(elem) > max:
    max = len(elem)

print(f'La unghezza massima è: {max}')

for i, row in enumerate(matrix):
  matrix[i] = matrix[i] + [''] * (max - len(row))

La unghezza massima è: 95


In [128]:
matrix = tf.convert_to_tensor(matrix)
#matrix = tf.convert_to_tensor(np.random.rand(3054, 95))

In [169]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=8000, # Dimensione dizionario
                                                  filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                                                  lower=True)

tokenizer.fit_on_texts(train['sentence'].values)
word_index = tokenizer.word_index
print(f'Trovati {len(word_index)} token univoci.')

Trovati 7577 token univoci.


In [171]:
X = tokenizer.texts_to_sequences(train['sentence'].values)
X = pad_sequences(X, maxlen=95, padding='post')
print('Shape of data tensor:', X.shape)

Shape of data tensor: (3054, 95)


How to find batch size, if it not belongs to N, the code will not work (we know why but is not crystal clear)

In [180]:
# define the LSTM model
model = keras.Sequential()
model.add(keras.layers.Input(shape=(95,)))
model.add(keras.layers.Lambda(lambda x: tf.reshape(x, [6, 95, 1])))
model.add(keras.layers.LSTM(64))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_8 (Lambda)            (6, 95, 1)                0         
_________________________________________________________________
lstm_22 (LSTM)               (6, 64)                   16896     
_________________________________________________________________
dropout_30 (Dropout)         (6, 64)                   0         
_________________________________________________________________
dense_30 (Dense)             (6, 32)                   2080      
_________________________________________________________________
dropout_31 (Dropout)         (6, 32)                   0         
_________________________________________________________________
dense_31 (Dense)             (6, 2)                    66        
Total params: 19,042
Trainable params: 19,042
Non-trainable params: 0
_________________________________________________

In [93]:
one_hot_train = tf.convert_to_tensor(
    OneHotEncoder(sparse=False).fit_transform(
        train.review_type.to_numpy().reshape(-1, 1)
        )
    )

one_hot_test = OneHotEncoder(sparse=False).fit_transform(
  test.review_type.to_numpy().reshape(-1, 1)
)

In [182]:
result = model.fit(X, one_hot_train, epochs=5, batch_size=6)

Epoch 1/5
509/509 [==============================] - 11s 21ms/step - loss: 0.6128 - accuracy: 0.7040
Epoch 2/5
509/509 [==============================] - 11s 21ms/step - loss: 0.6137 - accuracy: 0.7040
Epoch 3/5
509/509 [==============================] - 11s 21ms/step - loss: 0.6139 - accuracy: 0.7040
Epoch 4/5
509/509 [==============================] - 11s 21ms/step - loss: 0.6144 - accuracy: 0.7040
Epoch 5/5
509/509 [==============================] - 11s 21ms/step - loss: 0.6114 - accuracy: 0.7040
